In [ ]:
%load_ext autoreload
%autoreload 2

# JSON simulation language
For simple exchange of simulation descriptions a JSON format for the simulation was
developed.

In [1]:
import sbmlsim
print(sbmlsim.__version__)

0.0.6
